# yt-dev

## 2/16/17

Let's take T2Sampler2D for example. It is called in `pixelization_routines.pyx` by the `pixelize_element_mesh` function. The `pixelize_element_mesh` function is called in `cartesian_coordinates.py` by the `pixelize` function of the `CartesianCoordinateHandler` class. Here is backtrace for call to `pixelize` when asking for a `SlicePlot` from an exodus data set:
```
slc = yt.SlicePlot(ds, 'z', ('all','group1_lin'), origin="native")
  /home/al007/yt/yt/visualization/plot_window.py(2017) SlicePlot()
-> return AxisAlignedSlicePlot(ds, normal, fields, *args, **kwargs)
  /home/al007/yt/yt/visualization/plot_window.py(1334) AxisAlignedSlicePlot.__init__()
-> PWViewerMPL.__init__()
  /home/al007/yt/yt/visualization/plot_window.py(670) PWViewerMPL.__init__()
-> PlotWindow.__init__(self, *args, **kwargs)
  /home/al007/yt/yt/visualization/plot_window.py(223) PlotWindow.__init__()
-> self._setup_plots()
  /home/al007/yt/yt/visualization/plot_window.py(758) PWViewerMPL._setup_plots()
-> self._recreate_frb()
  /home/al007/yt/yt/visualization/plot_window.py(282) PlotWindow._recreate_frb()
-> self._frb._get_data_source_fields()
  /home/al007/yt/yt/visualization/fixed_resolution.py(156) FixedResolutionBuffer._get_data_source_fields()
-> self[f]
  /home/al007/yt/yt/visualization/fixed_resolution.py(135) FixedResolutionBuffer.__getitem__()
-> buff = self.ds.coordinates.pixelize(...)
> /home/al007/yt/yt/geometry/coordinates/cartesian_coordinates.py(68) CartesianCoordinatehandler.pixelize()
-> index = data_source.ds.index
```

Inheritance pattern for some of this plotting stuff:

AxisAlignedSlicePlot <- PWViewerMPL <- PlotWindow <- ImagePlotContainer <- object

WindowPlotMPL <- ImagePlotMPL <- PlotMPL <- object

In [8]:
import numpy as np

start_point = np.array([1, 1, 1])
end_point = np.array([2, 3, 4])
resolution = 10
lin_sample_points = np.zeros((resolution + 1, 3))
arc_length = np.zeros(resolution + 1)

lin_vec = end_point - start_point
lin_length = np.linalg.norm(lin_vec)
lin_inc = lin_vec / resolution
inc_length = np.linalg.norm(lin_inc)
lin_sample_points[0] = start_point
arc_length[0] = 0
for i in range(1, resolution + 1):
    lin_sample_points[i] = lin_sample_points[i-1] + lin_inc
    arc_length[i] = arc_length[i-1] + inc_length

print(lin_sample_points)
print(arc_length)

[[ 1.   1.   1. ]
 [ 1.1  1.2  1.3]
 [ 1.2  1.4  1.6]
 [ 1.3  1.6  1.9]
 [ 1.4  1.8  2.2]
 [ 1.5  2.   2.5]
 [ 1.6  2.2  2.8]
 [ 1.7  2.4  3.1]
 [ 1.8  2.6  3.4]
 [ 1.9  2.8  3.7]
 [ 2.   3.   4. ]]
[ 0.          0.37416574  0.74833148  1.12249722  1.49666295  1.87082869
  2.24499443  2.61916017  2.99332591  3.36749165  3.74165739]


In [9]:
print(lin_length)

3.74165738677
